In [ ]:
from pyspark import SparkContext
import pyspark

conf = pyspark.SparkConf().set("spark.driver.host", "localhost")
sc = SparkContext.getOrCreate()
sc.setLogLevel('OFF')

In [ ]:
supervisor_sf_pair = sc.textFile("../data/SF_business/supervisor_sf.tsv")\
                       .map(lambda x : x.split("\t"))\
                       .map(lambda x : (x[0], x[1]))

In [ ]:
zip_biz_pair = sc.textFile("../data/SF_business/filtered_registered_business_sf.tsv")\
                 .map(lambda x : x.split("\t"))\
                 .map(lambda x : (x[0], x[1:]))

## 1. Return any five distinct  zip code in zip_biz_pair

In [ ]:
zip_biz_pair.

## 2. Return any five distinct city names.

In [ ]:
zip_biz_pair.

## 3. Return five distinct (zip code, city name) pairs ordered by zip code, where zip code has 5 digits.

In [ ]:
distinct_zip_city_pairs = zip_biz_pair.

In [ ]:
distinct_zip_city_pairs.take(5)

## 4. Find the zip associated with the most city names
* which one is more efficient ??
    - zip_associated_with_multiple_cities_1  $vs$ zip_associated_with_multiple_cities_2

### Which one is more efficient? v1 or v2??

#### 

## 5. Create zip_city_count_names which includes (zip, (count, the list of names))

In [ ]:
zip_city_count_names = 

In [ ]:
zip_city_count_names.take(1)

## 6. convert zip_city_count_names to (zip, city_name)

In [ ]:
zip_city_count_names.

In [ ]:
sc.stop()